In [205]:
import os
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv
import panel as pn
from panel.interact import interact
from panel import widgets
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta
import requests
import json

%matplotlib inline

In [206]:
load_dotenv()

True

In [207]:
stock_key = os.getenv("FMP_API_KEY")

In [208]:
portfolio_df = pd.read_csv('../Desktop/TestPortfolio.csv')

In [209]:
portfolio_df

,Ticker,Transaction,Date,Transaction Price,Number of shares
0,AAPL,B,3/5/12,77.80,100
1,AAPL,B,4/1/13,60.57,100
2,TSLA,B,9/23/13,189.90,100
3,TSLA,S,11/18/13,122.38,75
4,TSLA,B,5/9/16,208.00,30
5,GOOG,B,8/7/17,915.39,10
6,AAPL,S,3/16/20,240.00,50


In [210]:
portfolio_df['Date']= pd.to_datetime(portfolio_df['Date'])

In [211]:
stock_dict_copy = {'AAPL': {'PE Ratio': 20.81351450883162,
  'Profit Margin': 0.21238094505984456,
  'Cash': 48844000000,
  'Beta': 1.228499,
  'Last Dividend': 3.08,
  'Industry': 'Computer Hardware',
  'Sector': 'Technology',
  'Daily Closing Price': [{'Date': '2012-03-05', 'Price': 352.84},
   {'Date': '2013-04-01', 'Price': 343.99}]}} 

In [212]:
date=[]
price=[]
stock_name=[]

for stock in stock_dict_copy:
    for d in stock_dict_copy[stock]['Daily Closing Price']:
        stock_name.append(stock)
        date.append(datetime.strptime(d['Date'],'%Y-%m-%d'))
        price.append(d['Price'])
        
print(date, price, stock_name)
closing_price_df= pd.DataFrame(list(zip(date, stock_name, price)), columns=['Date', 'Ticker', 'Close'])

[datetime.datetime(2012, 3, 5, 0, 0), datetime.datetime(2013, 4, 1, 0, 0)] [352.84, 343.99] ['AAPL', 'AAPL']


In [213]:
closing_price_df

,Date,Ticker,Close
0,2012-03-05,AAPL,352.84
1,2013-04-01,AAPL,343.99


In [214]:
portfolio_df.dtypes

Ticker                       object
Transaction                  object
Date                 datetime64[ns]
Transaction Price           float64
Number of shares              int64
dtype: object

In [215]:
portfolio_df = pd.merge(portfolio_df, closing_price_df, how='left', left_on=['Date', 'Ticker'], right_on=['Date', 'Ticker'])
portfolio_df.head()

,Ticker,Transaction,Date,Transaction Price,Number of shares,Close
0,AAPL,B,2012-03-05,77.80,100,352.84
1,AAPL,B,2013-04-01,60.57,100,343.99
2,TSLA,B,2013-09-23,189.90,100,NaN
3,TSLA,S,2013-11-18,122.38,75,NaN
4,TSLA,B,2016-05-09,208.00,30,NaN


In [216]:
portfolio_df['Net Shares'] = portfolio_df.apply(lambda row: row['Number of shares'] if row.Transaction == "B" else row['Number of shares'] * -1, axis=1)
portfolio_df

,Ticker,Transaction,Date,Transaction Price,Number of shares,Close,Net Shares
0,AAPL,B,2012-03-05,77.80,100,352.84,100
1,AAPL,B,2013-04-01,60.57,100,343.99,100
2,TSLA,B,2013-09-23,189.90,100,NaN,100
3,TSLA,S,2013-11-18,122.38,75,NaN,-75
4,TSLA,B,2016-05-09,208.00,30,NaN,30
5,GOOG,B,2017-08-07,915.39,10,NaN,10
6,AAPL,S,2020-03-16,240.00,50,NaN,-50


In [217]:
portfolio_df['Total Shares'] = portfolio_df[['Net Shares','Ticker']].groupby('Ticker').cumsum()
portfolio_df

,Ticker,Transaction,Date,Transaction Price,Number of shares,Close,Net Shares,Total Shares
0,AAPL,B,2012-03-05,77.80,100,352.84,100,100
1,AAPL,B,2013-04-01,60.57,100,343.99,100,200
2,TSLA,B,2013-09-23,189.90,100,NaN,100,100
3,TSLA,S,2013-11-18,122.38,75,NaN,-75,25
4,TSLA,B,2016-05-09,208.00,30,NaN,30,55
5,GOOG,B,2017-08-07,915.39,10,NaN,10,10
6,AAPL,S,2020-03-16,240.00,50,NaN,-50,150


In [218]:
portfolio_df['Daily Balance'] = portfolio_df['Close']*portfolio_df['Total Shares']
portfolio_df

,Ticker,Transaction,Date,Transaction Price,Number of shares,Close,Net Shares,Total Shares,Daily Balance
0,AAPL,B,2012-03-05,77.80,100,352.84,100,100,35284.0
1,AAPL,B,2013-04-01,60.57,100,343.99,100,200,68798.0
2,TSLA,B,2013-09-23,189.90,100,NaN,100,100,NaN
3,TSLA,S,2013-11-18,122.38,75,NaN,-75,25,NaN
4,TSLA,B,2016-05-09,208.00,30,NaN,30,55,NaN
5,GOOG,B,2017-08-07,915.39,10,NaN,10,10,NaN
6,AAPL,S,2020-03-16,240.00,50,NaN,-50,150,NaN


In [219]:
portfolio_df['Cost'] = pd.DataFrame(portfolio_df['Transaction Price'] * portfolio_df['Net Shares'])
portfolio_df

,Ticker,Transaction,Date,Transaction Price,Number of shares,Close,Net Shares,Total Shares,Daily Balance,Cost
0,AAPL,B,2012-03-05,77.80,100,352.84,100,100,35284.0,7780.0
1,AAPL,B,2013-04-01,60.57,100,343.99,100,200,68798.0,6057.0
2,TSLA,B,2013-09-23,189.90,100,NaN,100,100,NaN,18990.0
3,TSLA,S,2013-11-18,122.38,75,NaN,-75,25,NaN,-9178.5
4,TSLA,B,2016-05-09,208.00,30,NaN,30,55,NaN,6240.0
5,GOOG,B,2017-08-07,915.39,10,NaN,10,10,NaN,9153.9
6,AAPL,S,2020-03-16,240.00,50,NaN,-50,150,NaN,-12000.0


In [220]:
portfolio_df['Gains/Losses'] = portfolio_df['Daily Balance'] - portfolio_df['Cost']
portfolio_df

,Ticker,Transaction,Date,Transaction Price,Number of shares,Close,Net Shares,Total Shares,Daily Balance,Cost,Gains/Losses
0,AAPL,B,2012-03-05,77.80,100,352.84,100,100,35284.0,7780.0,27504.0
1,AAPL,B,2013-04-01,60.57,100,343.99,100,200,68798.0,6057.0,62741.0
2,TSLA,B,2013-09-23,189.90,100,NaN,100,100,NaN,18990.0,NaN
3,TSLA,S,2013-11-18,122.38,75,NaN,-75,25,NaN,-9178.5,NaN
4,TSLA,B,2016-05-09,208.00,30,NaN,30,55,NaN,6240.0,NaN
5,GOOG,B,2017-08-07,915.39,10,NaN,10,10,NaN,9153.9,NaN
6,AAPL,S,2020-03-16,240.00,50,NaN,-50,150,NaN,-12000.0,NaN


In [ ]:
portfolio_df.drop

In [231]:
gains_losses_df= portfolio_df.drop(columns=['Transaction', 'Transaction Price', 'Number of shares', 'Close', 'Net Shares', 'Total Shares'])
gains_losses_df.drop(columns=['Daily Balance', 'Cost'], inplace=True)
gains_losses_df.sort_values('Ticker').set_index(['Ticker', 'Date'])

Gains/Losses
Ticker Date                    
AAPL   2012-03-05       27504.0
       2013-04-01       62741.0
       2020-03-16           NaN
GOOG   2017-08-07           NaN
TSLA   2013-09-23           NaN
       2013-11-18           NaN
       2016-05-09           NaN

In [234]:
gains_losses_df.hvplot(x= 'index', label='gains by ticker, 2012-2020')

:Curve   [index]   (Gains/Losses)

In [91]:
daily_balance_df = portfolio_df.sort_values(['Ticker']).set_index(['Ticker', 'Date'])
daily_balance_df.drop(columns=['Transaction', 'Transaction Price', 'Number of shares', 'Close', 'Net Shares', 'Total Shares'], inplace=True)
daily_balance_df

Daily Balance     Cost
Ticker Date                              
AAPL   2012-03-05        35284.0   7780.0
       2013-04-01        68798.0   6057.0
       2020-03-16            NaN  12000.0
GOOG   2017-08-07            NaN   9153.9
TSLA   2013-09-23            NaN  18990.0
       2013-11-18            NaN   9178.5
       2016-05-09            NaN   6240.0

In [190]:
portfolio_df.dtypes

Ticker                object
Transaction           object
Date                  object
Transaction Price    float64
Number of shares       int64
Cost                 float64
dtype: object

In [191]:
portfolio_df['Date']= pd.to_datetime(portfolio_df['Date'])
portfolio_df

,Ticker,Transaction,Date,Transaction Price,Number of shares,Cost
0,AAPL,B,2012-03-05,77.80,100,7780.0
1,AAPL,B,2013-04-01,60.57,100,6057.0
2,TSLA,B,2013-09-23,189.90,100,18990.0
3,TSLA,S,2013-11-18,122.38,75,9178.5
4,TSLA,B,2016-05-09,208.00,30,6240.0
5,GOOG,B,2017-08-07,915.39,10,9153.9
6,AAPL,S,2020-03-16,240.00,50,12000.0


In [219]:
portfolio_df.set_index('Date')

,Ticker,Transaction,Transaction Price,Number of shares,Cost
Date,,,,,
2012-03-05,AAPL,B,77.80,100,7780.0
2013-04-01,AAPL,B,60.57,100,6057.0
2013-09-23,TSLA,B,189.90,100,18990.0
2013-11-18,TSLA,S,122.38,75,9178.5
2016-05-09,TSLA,B,208.00,30,6240.0
2017-08-07,GOOG,B,915.39,10,9153.9
2020-03-16,AAPL,S,240.00,50,12000.0


In [ ]:
portfolio_df['Total_share_by_date']

In [237]:
basis_df=pd.DataFrame(portfolio_df.groupby(["Date", "Ticker"])["Number of shares"].sum())
basis_df["Cost"]=portfolio_df.groupby(["Date", "Ticker"])["Cost"].sum()
basis_df = basis_df.reset_index() 
basis_df

,Date,Ticker,Number of shares,Cost
0,2012-03-05,AAPL,100,7780.0
1,2013-04-01,AAPL,100,6057.0
2,2013-09-23,TSLA,100,18990.0
3,2013-11-18,TSLA,75,9178.5
4,2016-05-09,TSLA,30,6240.0
5,2017-08-07,GOOG,10,9153.9
6,2020-03-16,AAPL,50,12000.0


In [236]:
basis_pivot["Shares Owned"]=basis_pivot["Number of shares"]["B"]-basis_pivot["Number of shares"]["S"]
basis_pivot["Contribution"]=basis_pivot["Cost"]["B"]-basis_pivot["Cost"]["S"]
basis_pivot.reset_index(inplace=True)
basis_pivot

index Ticker Number of shares           Cost           \
Transaction                             B     S        B        S   
0               0   AAPL            200.0  50.0  13837.0  12000.0   
1               1   GOOG             10.0   0.0   9153.9      0.0   
2               2   TSLA            130.0  75.0  25230.0   9178.5   

            Shares Owned Contribution  
Transaction                            
0                  150.0       1837.0  
1                   10.0       9153.9  
2                   55.0      16051.5

In [203]:
number_shares_df = portfolio_df.drop(columns=['Ticker', 'Transaction', 'Transaction Price', 'Cost'])
merged_df = pd.merge(closing_price_df, number_shares_df, how='inner', on='Date')
merged_df

,Date,Ticker 1 Closing Price,Ticker 2 Closing Price,Number of shares
0,2012-03-05,444,413,100
1,2013-04-01,220,498,100
2,2013-09-23,368,243,100
3,2013-11-18,303,372,75
4,2016-05-09,262,428,30
5,2017-08-07,239,460,10
6,2020-03-16,210,210,50


In [204]:
balance_ticker_1 = merged_df['Ticker 1 Closing Price']*merged_df['Number of shares']
balance_ticker_1 

0    44400
1    22000
2    36800
3    22725
4     7860
5     2390
6    10500
dtype: int64

In [205]:
balance_ticker_2 = merged_df['Ticker 2 Closing Price']*merged_df['Number of shares']
balance_ticker_2

0    41300
1    49800
2    24300
3    27900
4    12840
5     4600
6    10500
dtype: int64

In [210]:
balance_df_1 = pd.DataFrame(balance_ticker_1, columns=['Balance_Ticker_1'])
balance_df_2 = pd.DataFrame(balance_ticker_2, columns=['Balance_Ticker_2'])
balance_df_1

,Balance_Ticker_1
0,44400
1,22000
2,36800
3,22725
4,7860
5,2390
6,10500


In [211]:
balance_df_2

,Balance_Ticker_2
0,41300
1,49800
2,24300
3,27900
4,12840
5,4600
6,10500


In [214]:
relative_stock_balance = pd.concat([merged_df, balance_df_1, balance_df_2], axis='columns', join='outer')
relative_stock_balance

,Date,Ticker 1 Closing Price,Ticker 2 Closing Price,Number of shares,Balance_Ticker_1,Balance_Ticker_2
0,2012-03-05,444,413,100,44400,41300
1,2013-04-01,220,498,100,22000,49800
2,2013-09-23,368,243,100,36800,24300
3,2013-11-18,303,372,75,22725,27900
4,2016-05-09,262,428,30,7860,12840
5,2017-08-07,239,460,10,2390,4600
6,2020-03-16,210,210,50,10500,10500


,Ticker,Transaction,Number of shares,Cost
0,AAPL,B,200,13837.0
1,AAPL,S,50,12000.0
2,GOOG,B,10,9153.9
3,TSLA,B,130,25230.0
4,TSLA,S,75,9178.5


Number of shares           Cost         
Transaction                B     S        B        S
Ticker                                              
AAPL                   200.0  50.0  13837.0  12000.0
GOOG                    10.0   0.0   9153.9      0.0
TSLA                   130.0  75.0  25230.0   9178.5

In [195]:
basis_pivot["Shares Owned"]=basis_pivot["Number of shares"]["B"]-basis_pivot["Number of shares"]["S"]
basis_pivot["Contribution"]=basis_pivot["Cost"]["B"]-basis_pivot["Cost"]["S"]
basis_pivot.reset_index(inplace=True)
basis_pivot

Ticker Number of shares           Cost          Shares Owned  \
Transaction                       B     S        B        S                
0             AAPL            200.0  50.0  13837.0  12000.0        150.0   
1             GOOG             10.0   0.0   9153.9      0.0         10.0   
2             TSLA            130.0  75.0  25230.0   9178.5         55.0   

            Contribution  
Transaction               
0                 1837.0  
1                 9153.9  
2                16051.5

In [ ]:
for 'Date' in

In [196]:
def portfolio_performance():
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-196-4ef3130bbe28>, line 3)

In [ ]:
gain = 